In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np

In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Ilostat")

In [3]:
df = pd.read_csv("Ilostat_paises_selecionados_prop_publico.csv", sep=";", thousands=".", decimal=',')

In [20]:
df= df.rename(columns={" prop_vinculos_publicos ":'prop_vinculos_publicos',"País":'pais'})

In [24]:
df['pais'].unique()

array(['África do Sul', 'Argentina', 'Bolivia', 'Brasil', 'Chile',
       'Colombia', 'Dinamarca', 'Estados Unidos', 'França', 'México',
       'Peru', 'Suécia', 'Uruguai', 'OCDE'], dtype=object)

In [16]:
## Subindo para GBQ

client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao') # verificar a categoria do dado

In [21]:
df.columns

Index(['pais', 'fonte_pesquisa', 'ano_pesquisa', 'prop_vinculos_publicos'], dtype='object')

In [22]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
 bigquery.SchemaField('ano_pesquisa','Float',description='Ano de coleta da informação'),
 bigquery.SchemaField('prop_vinculos_publicos','Float',description='Proporção de vínculos públicos daquele país')
 ]

In [23]:
table_ref = dataset_ref.table('ILOSTAT_proporcao_publicos_selecionados') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=3b259e6d-c5a8-436f-9de4-62bfe25ab9ac>